# Main App

In [1507]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1508]:
#|export
import pandas as pd
from flask import Flask, redirect, request, render_template, flash
from dataclasses import dataclass
import re

In [1509]:

from IPython.display import display_html

In [1510]:
def create_db():
    people = [
        {
            "firstname": "John",
            "lastname": "Doe",
            "phone": "555-1234",
            "email": "john.doe@example.com"
        },
        {
            "firstname": "Jane",
            "lastname": "Smith",
            "phone": "555-5678",
            "email": "jane.smith@example.com"
        },
        {
            "firstname": "Alice",
            "lastname": "Johnson",
            "phone": "555-8765",
            "email": "alice.johnson@example.com"
        },
        {
            "firstname": "Bob",
            "lastname": "Williams",
            "phone": "555-4321",
            "email": "bob.williams@example.com"
        },
        {
            "firstname": "Charlie",
            "lastname": "Brown",
            "phone": "555-9876",
            "email": "charlie.brown@example.com"
        },
        {
            "firstname": "Diana",
            "lastname": "Miller",
            "phone": "555-6543",
            "email": "diana.miller@example.com"
        },
        {
            "firstname": "Edward",
            "lastname": "Davis",
            "phone": "555-3456",
            "email": "edward.davis@example.com"
        },
        {
            "firstname": "Fiona",
            "lastname": "Garcia",
            "phone": "555-6789",
            "email": "fiona.garcia@example.com"
        },
        {
            "firstname": "George",
            "lastname": "Martinez",
            "phone": "555-7890",
            "email": "george.martinez@example.com"
        },
        {
            "firstname": "Hannah",
            "lastname": "Rodriguez",
            "phone": "555-8901",
            "email": "hannah.rodriguez@example.com"
        }
    ]

    # Writing the JSON data to a file
    file_path = 'contactdb.json'
    # df = pd.read_json(file_path)
    df = pd.DataFrame()
    df = df.from_dict(people)
    df['id'] = df.index
    df.to_json(file_path, orient='records')

In [1511]:
create_db()
db = pd.read_json('contactdb.json')
db.to_dict('records')

[{'firstname': 'John',
  'lastname': 'Doe',
  'phone': '555-1234',
  'email': 'john.doe@example.com',
  'id': 0},
 {'firstname': 'Jane',
  'lastname': 'Smith',
  'phone': '555-5678',
  'email': 'jane.smith@example.com',
  'id': 1},
 {'firstname': 'Alice',
  'lastname': 'Johnson',
  'phone': '555-8765',
  'email': 'alice.johnson@example.com',
  'id': 2},
 {'firstname': 'Bob',
  'lastname': 'Williams',
  'phone': '555-4321',
  'email': 'bob.williams@example.com',
  'id': 3},
 {'firstname': 'Charlie',
  'lastname': 'Brown',
  'phone': '555-9876',
  'email': 'charlie.brown@example.com',
  'id': 4},
 {'firstname': 'Diana',
  'lastname': 'Miller',
  'phone': '555-6543',
  'email': 'diana.miller@example.com',
  'id': 5},
 {'firstname': 'Edward',
  'lastname': 'Davis',
  'phone': '555-3456',
  'email': 'edward.davis@example.com',
  'id': 6},
 {'firstname': 'Fiona',
  'lastname': 'Garcia',
  'phone': '555-6789',
  'email': 'fiona.garcia@example.com',
  'id': 7},
 {'firstname': 'George',
  'last

In [1512]:
q = '1234'
# db[db['firstname'].str.lower().str.contains(q)]
predicate_firstname = db['firstname'].str.lower().str.contains(q.lower())
predicate_lastname = db['lastname'].str.lower().str.contains(q.lower())
predicate_phone = db['phone'].str.lower().str.contains(q.lower())
predicate_email= db['email'].str.lower().str.contains(q.lower())
db[predicate_firstname|predicate_lastname|predicate_phone|predicate_email]

,firstname,lastname,phone,email,id
0,John,Doe,555-1234,john.doe@example.com,0


In [1513]:
db.columns

a={'firstname': 'Rahul',
   'lastname': 'Saraf',
    'phone':'7347399',
    'email': 'rahuketu86'}
a['id'] = len(db)
# db.loc[len(db)] = 
a
db.loc[len(db), a.keys()] = a.values()

In [1514]:
#|export
class ContactErrors:
    firstname:str = "Error in Firstname"
    lastname:str = "Error in Lastname"
    phone:str = "Error in Phone"
    email:str = "Error in Email"

In [1515]:
ContactErrors()

In [1465]:
#|export

# Regular expression pattern for validating email
email_regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
phone_regex =  r'^(?:\d{7,}|\d{1,}-\d{1,}-?\d{5,}|\d+-\d{7,}|\d{3}-\d{4})$'

In [1466]:
#|export

# Function to validate email
def validate_email(email):
    if re.match(email_regex, email): return True
    else: return False

In [1467]:
#|export

# Function to validate phone number
def validate_phone_number(phone_number):
    if re.match(phone_regex, phone_number): return True
    else: return False

In [1468]:

# Example usage
phone_numbers = ["555-1234", "917347306788", "7237424", "123-456", "123-456-7890", "123--456", "123-45"]
for number in phone_numbers:
    if validate_phone_number(number):
        print(f"{number}: Valid phone number")
    else:
        print(f"{number}: Invalid phone number")

555-1234: Valid phone number
917347306788: Valid phone number
7237424: Valid phone number
123-456: Invalid phone number
123-456-7890: Invalid phone number
123--456: Invalid phone number
123-45: Invalid phone number


In [1469]:

# Example usage
email = "example@example.com"
email = "rahuketu"
if validate_email(email):
    print("Valid email")
else:
    print("Invalid email")

Invalid email


In [1470]:
#|export
@dataclass
class Contact:
    firstname:str=None
    lastname:str=None
    phone:str=None
    email:str=None
    id: int =None
    errors:ContactErrors = ContactErrors()

    def from_contacts_dict(self, c):
        self.firstname = c['firstname']
        self.lastname = c['lastname']
        self.phone = c['phone']
        self.email = c['email']
        self.id = c['id']

    def save(self):
        if not self.validate(): return False
        else: 
            db = Contacts()
            db.add(firstname = self.firstname, 
                   lastname = self.lastname, 
                   phone=self.phone, 
                   email=self.email)
            return True
        
    def edit(self):
        if not self.validate(): return False
        else:
            db = Contacts()
            db.edit(id=self.id,
                    firstname = self.firstname, 
                    lastname = self.lastname, 
                    phone=self.phone, 
                    email=self.email)
            return True


    def validate(self):
        if not self._check_nones([self.firstname, self.lastname, self.phone, self.email]): return False
        elif not validate_email(self.email): return False
        elif not validate_phone_number(self.phone): return False
        else:
            # print("Validated") 
            return True

    def _check_nones(self, ls):
        for e in ls: 
            # print(e)
            if e is None or e == "": 
                # print(e, "Noness or empty")
                return False
        return True
        

In [1471]:
Contact()

Contact(firstname=None, lastname=None, phone=None, email=None, id=None, errors=<__main__.ContactErrors object at 0x126f483e0>)

In [1516]:
#|export
class Contacts(object):
    def __init__(self) -> None:
        self.refresh()

    def search(self, q):
        predicate_firstname = self.db['firstname'].str.lower().str.contains(q.lower())
        predicate_lastname = self.db['lastname'].str.lower().str.contains(q.lower())
        predicate_phone = self.db['phone'].str.lower().str.contains(q.lower())
        predicate_email= self.db['email'].str.lower().str.contains(q.lower())
        filter_df = self.db[predicate_firstname|predicate_lastname|predicate_phone|predicate_email]
        return filter_df.to_dict('records')
    
    def refresh(self):
        self.file_path = 'contactdb.json'
        self.db = pd.read_json(self.file_path)

    def all(self):
        return self.db.to_dict('records')
    
    def get(self, id):
        contact = self.db[self.db['id']==id]
        # print(contact)
        return contact.to_dict('records')[0]
    
    def add(self, firstname, lastname, phone, email):
        a={'firstname': firstname,
           'lastname': lastname,
           'phone':phone,
           'email': email}
        a['id'] = self.db['id'].max()+1
        self.db.loc[len(self.db), a.keys()] = a.values()
        self.db.to_json(self.file_path, orient='records')
        self.refresh()

    def delete(self, id):
        id_idx = self.db[self.db['id']==id].index
        self.db.drop(index=id_idx, inplace=True, errors='raise')
        self.db.to_json(self.file_path, orient='records')
        self.refresh()

    def edit(self, id, firstname, lastname, phone, email):
        a={'firstname': firstname,
           'lastname': lastname,
           'phone':phone,
           'email': email,
           'id': id}
        id_idx = self.db[self.db['id']==id].index
        self.db.loc[id_idx, a.keys()] = a.values()
        self.db.to_json(self.file_path, orient='records')
        self.refresh()


In [1517]:
c_set = Contacts()
c_dict = c_set.get(0)
c_dict
c = Contact()
c.from_contacts_dict(c_dict)
c.phone = '555-1001'
c.edit()

True

In [1518]:
# new_vals = [2, 'Alice', 'Johnson', '555-8767', 'alice.johnson@example.com']
# c.edit(*new_vals)
# c.get(2)

In [1519]:
c = Contacts()
a_dict = c.db.loc[0].to_dict()
a_dict.pop('id', None); a_dict
c.delete(0)

c.add(*list(a_dict.values()))
c.all()




[{'firstname': 'Jane',
  'lastname': 'Smith',
  'phone': '555-5678',
  'email': 'jane.smith@example.com',
  'id': 1},
 {'firstname': 'Alice',
  'lastname': 'Johnson',
  'phone': '555-8765',
  'email': 'alice.johnson@example.com',
  'id': 2},
 {'firstname': 'Bob',
  'lastname': 'Williams',
  'phone': '555-4321',
  'email': 'bob.williams@example.com',
  'id': 3},
 {'firstname': 'Charlie',
  'lastname': 'Brown',
  'phone': '555-9876',
  'email': 'charlie.brown@example.com',
  'id': 4},
 {'firstname': 'Diana',
  'lastname': 'Miller',
  'phone': '555-6543',
  'email': 'diana.miller@example.com',
  'id': 5},
 {'firstname': 'Edward',
  'lastname': 'Davis',
  'phone': '555-3456',
  'email': 'edward.davis@example.com',
  'id': 6},
 {'firstname': 'Fiona',
  'lastname': 'Garcia',
  'phone': '555-6789',
  'email': 'fiona.garcia@example.com',
  'id': 7},
 {'firstname': 'George',
  'lastname': 'Martinez',
  'phone': '555-7890',
  'email': 'george.martinez@example.com',
  'id': 8},
 {'firstname': 'Ha

In [1520]:
#|export
app = Flask(__name__)
app.secret_key = "superdupersecret"


In [1521]:
#|export
@app.get("/")
def index():
    return redirect("/contacts")

In [1522]:
#|export
@app.get("/contacts")
def contacts():
    search = request.args.get("q")
    contact_set = None
    if search is not None: contact_set = Contacts().search(search)
    else: contact_set = Contacts().all()
    # print(contact_set)
    return render_template("index.html", contact_set=contact_set)

In [1523]:
df = Contacts().db
df = df[df['id']==1].to_dict('records')[0]; df

{'firstname': 'Jane',
 'lastname': 'Smith',
 'phone': '555-5678',
 'email': 'jane.smith@example.com',
 'id': 1}

In [1524]:
#|export
@app.get("/contacts/<int:id>")
def view(id:int):
    return render_template("view.html", contact=Contacts().get(id))

In [1525]:
with app.app_context():
    print(view(2))

<!doctype html>
<html>
    <head>
        <title>Contact.App</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@picocss/pico@2/css/pico.min.css">
    </head>
    <body>
        <main class="container">
            <h1> CONTACTS.APP</h1>
            <h2> A demo contact application </h2>
            <hr>
            
    <article>
        <h1>Alice Johnson </h1>
        <div role="group">
            <div>555-8765</div>
            <div>alice.johnson@example.com</div>
        </div>
        <footer role="group">
            <a href="/contacts" role="button">Back</a>
            <a href="/contacts/2/edit" role="button">Edit</a>
        </footer>
    </article>
 
        </main>
    </body>
</html>


In [1526]:
Contacts().get(2)

{'firstname': 'Alice',
 'lastname': 'Johnson',
 'phone': '555-8765',
 'email': 'alice.johnson@example.com',
 'id': 2}

In [1527]:
#|export
@app.route("/contacts/<int:id>/edit", methods=['GET', 'POST'])
def edit(id):
    if request.method == 'GET':
        c_dict = Contacts().get(id)
        return render_template('edit.html', contact=c_dict)
    else:
        c_dict = Contacts().get(id)
        c = Contact()
        c.from_contacts_dict(c_dict)
        c.firstname = request.form['firstname']
        c.lastname = request.form['lastname']
        c.phone=request.form['phone']
        c.email=request.form['email']
        if c.edit():
            flash("Updated Contract")
            return redirect("/contacts/"+str(id))
        else: return render_template('edit.html', contact=c_dict)

In [1528]:
#|export
@app.route("/contacts/<int:id>/delete", methods=['POST'])
def delete(id):
    Contacts().delete(id)
    return redirect("/contacts")

In [1529]:
contact=Contact(firstname=None, lastname=None, phone=None, email=None)
contact.firstname

In [1530]:
#|export
@app.route("/contacts/new", methods=['GET'])
def contact_new_get():
    return render_template('new.html', contact=Contact(firstname=None, lastname=None, phone=None, email=None))

In [1531]:
#|export
@app.route("/contacts/new", methods=['POST'])
def contact_new():
    c = Contact(firstname=request.form['firstname'], 
                lastname=request.form['lastname'],
                phone=request.form['phone'],
                email=request.form['email'])
    if c.save():
        flash("New contract created")
        return redirect("/contacts")
    else: return render_template('new.html', contact=c)

In [1532]:
#|eval:false
from nbdev.export import nb_export
nb_export("01_main.ipynb", lib_path=".", name='main')